In [1]:
from transformers import BertModel, BertTokenizer
import torch

# Загрузка предобученной модели и токенизатора BERT
model_name = 'bert-base-uncased'  # можно выбрать другие модели, например, 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
import pandas as pd

In [6]:
dt = pd.read_excel(r"/content/sample_data/Регламент по пунктам AVAS (1).xlsx")

In [7]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        # Используем эмбеддинг токена [CLS], который представляет всё предложение
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()

In [19]:
target_text = 'The driver is in the Vehicle, the Vehicle is in the R drive mode and reversing at any speed, an external soundtrack notifies surrounding road users about the movement of the Vehicle in reverse, so that an approaching Vehicle can be identified by ear. '
target_embedding = get_embedding(target_text)

In [20]:
dt['emb'] = dt['текст'].apply(lambda x: get_embedding(x,))

In [21]:
dt

,Номер,Пункт,подпунтк,под-подпункт,Название,текст,emb,similarity
0,6,0,0,0,Specifications,Specifications,"[[-0.23776981, 0.28514847, -0.045147166, -0.09...",0.758350
1,6,1,0,0,General specifications,"For the purpose of this Regulation, the vehic...","[[-0.057323102, 0.14672153, 0.34528777, -0.119...",0.725245
2,6,2,0,0,Acoustics characteristics,The sound emitted by the vehicle type submitte...,"[[-0.5289814, -0.16232514, 0.581422, -0.419398...",0.627136
3,6,2,1,0,Constant speed tests,Constant speed tests,"[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.759435
4,6,2,1,1,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.37355855, 0.054526452, 0.6671835, -0.0714...",0.743258
5,6,2,1,2,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.5071613, 0.035247777, 0.03517357, -0.1444...",0.678074
6,6,2,1,3,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.23051897, -0.05120594, -0.61029536, -0.15...",0.668625
7,6,2,2,0,Reversing test,Reversing test,"[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.768305
8,6,2,2,1,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.43800455, 0.1940213, 0.49871156, -0.47878...",0.742176
9,6,2,2,2,Reversing test,If after a vehicle is tested in accordance wit...,"[[-0.16746762, -0.07895926, -0.47101647, -0.11...",0.690219


In [22]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
def calculate_cosine_similarity(embedding,target_embedding_2d):
    return cosine_similarity(embedding, target_embedding_2d).flatten()[0]

In [24]:
dt['similarity'] = dt['emb'].apply(lambda x: calculate_cosine_similarity(x,target_embedding))

In [25]:
dt

,Номер,Пункт,подпунтк,под-подпункт,Название,текст,emb,similarity
0,6,0,0,0,Specifications,Specifications,"[[-0.23776981, 0.28514847, -0.045147166, -0.09...",0.750294
1,6,1,0,0,General specifications,"For the purpose of this Regulation, the vehic...","[[-0.057323102, 0.14672153, 0.34528777, -0.119...",0.762362
2,6,2,0,0,Acoustics characteristics,The sound emitted by the vehicle type submitte...,"[[-0.5289814, -0.16232514, 0.581422, -0.419398...",0.697250
3,6,2,1,0,Constant speed tests,Constant speed tests,"[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.787975
4,6,2,1,1,Constant speed tests,The test speeds for approval are 10 km/h and 2...,"[[-0.37355855, 0.054526452, 0.6671835, -0.0714...",0.783343
5,6,2,1,2,Constant speed tests,When tested under the conditions of Annex 3 pa...,"[[-0.5071613, 0.035247777, 0.03517357, -0.1444...",0.741598
6,6,2,1,3,Constant speed tests,If after a vehicle is tested in accordance wit...,"[[-0.23051897, -0.05120594, -0.61029536, -0.15...",0.726869
7,6,2,2,0,Reversing test,Reversing test,"[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.778054
8,6,2,2,1,Reversing test,When tested under the conditions of Annex 3 pa...,"[[-0.43800455, 0.1940213, 0.49871156, -0.47878...",0.786192
9,6,2,2,2,Reversing test,If after a vehicle is tested in accordance wit...,"[[-0.16746762, -0.07895926, -0.47101647, -0.11...",0.754457
